# Modeling

In [1]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import MinTrace
from hierarchicalforecast.utils import aggregate

from valuation.infra.store.dataset import DatasetStore
from valuation.asset.identity.dataset import DatasetID
from valuation.core.stage import DatasetStage


/home/john/anaconda3/envs/valuation/lib/python3.12/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore
/home/john/anaconda3/envs/valuation/lib/python3.12/site-packages/statsforecast/core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
/home/john/anaconda3/envs/valuation/lib/python3.12/site-packages/statsforecast/utils.py:237: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


## Training Data

In [2]:
store = DatasetStore()
dataset_id = DatasetID(name="train_val", stage=DatasetStage.MODEL)
passport = store.get_passport(dataset_id=dataset_id)
ds = store.get(passport=passport)
train_df = ds.data

2025-10-23 13:20:24.504 | DEBUG    | valuation.asset.dataset.base:load:337 - Dataset Dataset train_val of the model stage created on 2025-10-23 at 12:45 loaded.


## Define the Model
We tell AutoARIMA to look for annual seasonality (season_length=52)

In [3]:
models = [AutoARIMA(season_length=52)]

## StatsForest Model

In [ ]:
sf = StatsForecast(    
    models=models,
    freq='W', 
    n_jobs=-1 # Use all cores
)

/home/john/anaconda3/envs/valuation/lib/python3.12/site-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(


## Blocked Cross-Validation
This generates the unreconciled forecasts for each fold. We must add fitted=True to get the in-sample forecasts for the reconciler.

In [ ]:
cv_df_base = sf.cross_validation(
    df=train_df,
    h=52,            # 52-week forecast horizon
    n_windows=5,     # 5 rolling folds
    fitted=True      # CRITICAL: This is required for reconciliation
)

/home/john/anaconda3/envs/valuation/lib/python3.12/site-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
Cross Validation Time Series 10:  20%|██        | 1/5 [07:41<30:46, 461.55s/it]

## Fit the Model

In [ ]:
sf.fit(df=train_df)

## In-Sample Fitted Values
Get the in-sample (training set) fitted values. The reconciler needs these to compute the error covariance.

In [ ]:
fitted_df = sf.forecast_fitted_values()

## Create Summing Matrix and Tags

In [ ]:
hierarchy_df = train_df[['unique_id', 'ds']].drop_duplicates()
hierarchy_df['store'] = hierarchy_df['unique_id'].apply(lambda s: s.split('_')[0])
hierarchy_df['category'] = hierarchy_df['unique_id'].apply(lambda s: s.split('_')[1])

# Define the hierarchy levels for the `aggregate` function
spec = [
    ['store'],
    ['category'],
    ['store', 'category'] # This is the bottom level
]

# The aggregate function returns:
# Y_hier_df: The aggregated dataframe (we don't need this)
# S_df: The Summing Matrix as a DataFrame
# tags: The tags dictionary (we also pass this to `reconcile`)
_, S_df, tags = aggregate(df=hierarchy_df, spec=spec)

## Reconciler for CV Forecasts

In [ ]:
reconcilers = [MinTrace(method='mint_ols')]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

In [ ]:
cv_df_reconciled = hrec.reconcile(
    Y_hat_df=cv_df_base,  # The forecasts from cross_validation
    Y_df=fitted_df,       # The in-sample fitted values
    S_df=S_df,            # The Summing Matrix
    tags=tags             # The hierarchy tags
)

## Evaluate CV Performance

In [ ]:
# Merge the 'y' (actual) values from the base CV output
actuals_df = cv_df_base[['unique_id', 'ds', 'cutoff', 'y']]
cv_df_eval = cv_df_reconciled.merge(
    actuals_df,
    on=['unique_id', 'ds', 'cutoff']
)

# Now, group by model name to compare
performance = cv_df_eval.groupby('model').agg(
    RMSE=pd.NamedAgg(column='y', aggfunc=rmse),
    MAE=pd.NamedAgg(column='y', aggfunc=mae)
)
print("--- Cross-Validation Performance ---")
print(performance)